<a href="https://colab.research.google.com/github/Rblivingstone/deep_learning/blob/master/GPT-2-mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import numpy as np

class CorpusGenerator:
    def __init__(self,start=46499973,max=5):
        self.n = start
        self.max = max
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
      if self.max!=None:
        if self.max>=self.i:
          result = self.get()
          self.n+=1
          self.i+=1
          return result
        else:
          raise StopIteration
      else:
        result = self.get()
        self.n+=1
        self.i+=1
        return result
    
    def get(self):
        resp = requests.get('https://www.ksl.com/article/{0}'.format(self.n))
        
        soup = BeautifulSoup(resp.content,'html.parser')
        text = soup.find("article",{"id":"kslMainArticle"}).get_text()
        try:
          cleaned_text = re.findall('(?<=— )(.*)(?=Copyright)',text)[0]
          return cleaned_text
        except IndexError:
          return("No Text Here")
        
class SequenceGenerator:
    def __init__(self,cg_max=10):
        self.tk = Tokenizer()
        self.cg = CorpusGenerator(max=cg_max)
        self.cg.n=46499973
        self.cg.i=0
        self.tk.fit_on_texts(CorpusGenerator(max=cg_max))
        self.n = 0
        self.i = 0
        self.seqs = self.tk.texts_to_sequences(self.cg)
        self.vocab_size = len(self.tk.word_index) + 1
    
    def __iter__(self):
        return self
  
    def __next__(self):
        if self.i+20<len(self.seqs[self.n])-1:
            output = (np.array(self.seqs[self.n][self.i:self.i+20]).reshape(1,20),
                      to_categorical(self.seqs[self.n][self.i+20],
                                     num_classes=self.vocab_size).reshape(1,self.vocab_size))
            self.i+=1
            return output
        else:
            self.n += 1
            self.i = 0
            NoData = True
            while NoData:
              if len(self.seqs[self.n])<21:
                self.n+=1
              else:
                NoData=False
            if self.n>len(self.seqs)-1:
              self.n=0
            output = (np.array(self.seqs[self.n][self.i:self.i+20]).reshape(1,20),
                      to_categorical(self.seqs[self.n][self.i+20],
                                     num_classes=self.vocab_size).reshape(1,self.vocab_size))
            self.i+=1
            return output
    
    

Using TensorFlow backend.


In [0]:
sg = SequenceGenerator(cg_max=100)

In [3]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

vocab_size = sg.vocab_size

model = Sequential()
model.add(Embedding(vocab_size, 5000, input_length=20))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(5000, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 5000)          23570000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 100)           2040400   
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 100)           80400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 20, 100)           80400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 20, 100)           80400     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dens

In [0]:
model.fit_generator(sg,steps_per_epoch=10000,epochs=1000)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
  466/10000 [>.............................] - ETA: 2:12:11 - loss: 7.3505 - acc: 0.0536

In [0]:
sg.seqs